In [37]:
import pandas as pd
X = pd.DataFrame({'age':[25, 32, 47, 51, 62],
                  'height':[182, 176, 174, 168, 181],
                  'weight': [75, 71, 78, 72, 86]})
X

,age,height,weight
0,25,182,75
1,32,176,71
2,47,174,78
3,51,168,72
4,62,181,86


In [38]:
y=pd.Series([20, 32, 45, 55, 61],name='income')
y

0    20
1    32
2    45
3    55
4    61
Name: income, dtype: int64

In [39]:
import numpy as np
### make X_shadow by randomly permuting each column of X
np.random.seed(42)
X_shadow = X.apply(np.random.permutation)
X_shadow.columns = ['shadow_' + feat for feat in X.columns]
### make X_boruta by appending X_shadow to X
X_boruta = pd.concat([X, X_shadow], axis = 1)
X_boruta

,age,height,weight,shadow_age,shadow_height,shadow_weight
0,25,182,75,32,168,71
1,32,176,71,62,176,75
2,47,174,78,47,174,72
3,51,168,72,25,182,86
4,62,181,86,51,181,78


In [40]:
from sklearn.ensemble import RandomForestRegressor

### fit a random forest (suggested max_depth between 3 and 7)
forest = RandomForestRegressor(max_depth = 5, random_state = 42)
forest.fit(X_boruta, y)

### store feature importances
feat_imp_X = forest.feature_importances_[:len(X.columns)]
feat_imp_shadow = forest.feature_importances_[len(X.columns):]

### compute hits
hits = feat_imp_X > feat_imp_shadow.max()
hits

array([ True, False, False])

In [41]:
feat_imp = pd.concat([pd.Series(feat_imp_X.tolist()), pd.Series(feat_imp_shadow.tolist())], axis = 0)
feat_imp

0    0.281124
1    0.158089
2    0.092546
0    0.101621
1    0.211820
2    0.154799
dtype: float64

In [42]:
### initialize hits counter
hits = np.zeros((len(X.columns)))
### repeat 20 times
for iter_ in range(20):
   ### make X_shadow by randomly permuting each column of X
   np.random.seed(iter_)
   X_shadow = X.apply(np.random.permutation)
   X_boruta = pd.concat([X, X_shadow], axis = 1)
   ### fit a random forest (suggested max_depth between 3 and 7)
   forest = RandomForestRegressor(max_depth = 5, random_state = 42)
   forest.fit(X_boruta, y)
   ### store feature importance
   feat_imp_X = forest.feature_importances_[:len(X.columns)]
   feat_imp_shadow = forest.feature_importances_[len(X.columns):]
   ### compute hits for this trial and add to counter
   hits += (feat_imp_X > feat_imp_shadow.max())
print(hits)

[18.  5.  0.]


In [43]:
import scipy

trials = 20
pmf = [scipy.stats.binom.pmf(x, trials, .5) for x in range(trials + 1)]
pmf

[9.5367431640625e-07,
 1.9073486328125e-05,
 0.0001811981201171875,
 0.0010871887207031235,
 0.004620552062988286,
 0.014785766601562505,
 0.03696441650390626,
 0.07392883300781253,
 0.12013435363769544,
 0.16017913818359386,
 0.1761970520019531,
 0.16017913818359392,
 0.12013435363769544,
 0.07392883300781251,
 0.03696441650390626,
 0.014785766601562505,
 0.004620552062988286,
 0.0010871887207031233,
 0.00018119812011718745,
 1.9073486328124997e-05,
 9.5367431640625e-07]

In [44]:
# !pip install boruta

In [59]:
from boruta import BorutaPy
from sklearn.ensemble import RandomForestRegressor
import numpy as np
###initialize Boruta
forest = RandomForestRegressor(
   n_jobs = -1, 
   max_depth = 5
)

boruta = BorutaPy(
   estimator = forest, 
   n_estimators = 'auto',
   max_iter = 100 # number of trials to perform
)
print(np.array(X).dtype)
print(np.array(y).dtype)

### fit Boruta (it accepts np.array, not pd.DataFrame)
boruta.fit(np.array(X), np.array(y))

### print results
green_area = X.columns[boruta.support_].to_list()
blue_area = X.columns[boruta.support_weak_].to_list()
print('features in the green area:', green_area)
print('features in the blue area:', blue_area)

int64
int64


NameError: name 'int_' is not defined